# Blurred Ball Cover

In [72]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import random
from collections import namedtuple
import numpy.linalg as npl
from sets import Set

In [73]:
Ball = namedtuple("Ball", ["center", "radius"])
BlurredBall = namedtuple("BlurredBall", ["k", "MEB"])

In [74]:
def approx_meb_backup(points, epsilon):
    coreset = []
    num_of_points = len(points)
    furthest_point = None
    center = points[0]
    #print("Points: ", points)
    #print("Initial Center: " , points[0][0], points[0][1])
    num_iterations = int(1.0 / (epsilon * epsilon))
    #num_iterations = 1000
    #print("Number of Iterations: " + repr(num_iterations))
    distances = {}

    prev_radius = -10000.0
    
    for i in range(1, num_iterations):
        #print("Iteration: ", i)
        for j in range(num_of_points):
            distances[j] = npl.norm(points[j] - center)
        furthest_point_index = max(distances, key = distances.get)
        furthest_point = points[furthest_point_index]
        furthest_distance = distances[furthest_point_index]

        center = center + (1.0 / i) * (furthest_point - center)

        if abs(furthest_distance - prev_radius) < 0.01:
            break
            
    coreset_size = 1.0 / epsilon
    # Pick (1/epsilon)
    for point in points:
        if abs(npl.norm(point - center) - furthest_distance) < 0.01:
            coreset.append(point)
            
    radius = npl.norm(furthest_point - center)
    blurred_ball = BlurredBall(coreset, Ball(center, radius))
    return blurred_ball

In [75]:
def generate_normal_data(num_of_points, num_of_dimensions):
    mean = random.sample(np.array(range(100)), num_of_dimensions)
    scalar_variances = 10.0 * np.ones(num_of_dimensions)
    covariance = np.diag(scalar_variances)
    points = np.random.multivariate_normal(mean, covariance, num_of_points)
    return points

def generate_multivariate_data(num_of_points, num_of_dimensions):
    probabilities = [float(1.0 / num_of_dimensions)] * num_of_dimensions
    points = np.random.multinomial(20, probabilities, size = num_of_points)
    return points


In [76]:
def inside_ball_eps(blurred_ball, point, epsilon):
    #print "inside_ball"
    center, radius = blurred_ball.MEB
    distance = npl.norm(point - center)
    if distance <= (1 + epsilon) * radius:
        return True
    else:
        return False

In [77]:
def approx_meb_light(points, epsilon):
    coreset = []
    num_of_points = len(points)
    furthest_point = None
    center = points[0]
    radius = 0.0
    num_iterations = int(1.0 / (epsilon * epsilon))
    distances = {}
    prev_radius = -10000.0
    coreset = [points[0]]
    for i in range(1, num_iterations):
        for j in range(num_of_points):
            distances[j] = npl.norm(points[j] - center)
        furthest_point_index = max(distances, key = distances.get)
        furthest_point = points[furthest_point_index]
        furthest_distance = distances[furthest_point_index]

        center = center + (1.0 / i) * (furthest_point - center)   
        if abs(furthest_distance - prev_radius) < 0.01:
            break
        prev_radius = furthest_distance
            
    radius = npl.norm(furthest_point - center)
    meb = Ball(center, radius)
    return meb

In [78]:
def epsilon_coreset(points, epsilon):
    coreset = []
    num_of_points = len(points)
    furthest_point = None
    center = points[0]
    radius = 0.0

    num_iterations = int(2.0 / (epsilon))
    print("Total Iterations: " + repr(num_iterations))
    distances = {}
    coreset_indices = Set()

    prev_radius = -10000.0
    
    coreset = [points[0]]
    for i in range(1, num_iterations):
        #print("Iteration: ", i, " radius: ", prev_radius)
        # Find furthest point
        for j in range(num_of_points):
            distances[j] = npl.norm(points[j] - center)
        # Calculated furthest point    
        furthest_point_index = max(distances, key = distances.get)
        furthest_point = points[furthest_point_index]
        furthest_distance = distances[furthest_point_index]
        if furthest_point_index in coreset_indices:
            continue
        coreset_indices.add(furthest_point_index)
        coreset.append(furthest_point)
        coreset_meb = approx_meb_light(coreset, 0.01)
        center = coreset_meb.center
        
    return coreset

In [79]:
def approx_meb(points, epsilon):
    coreset = epsilon_coreset(points, epsilon)
    meb = approx_meb_light(coreset, epsilon)
    return BlurredBall(coreset, meb)
    

In [80]:
def update(blurred_balls, A, epsilon):
    print "update"
    K = []
    for blurred_ball in blurred_balls:
        K.extend(blurred_ball.k)


    outer_loop_flag = False    
    for point in A:
        if outer_loop_flag == True:
            break
        for blurred_ball in blurred_balls:
            if inside_ball_eps(blurred_ball, point, epsilon) == False:
                outer_loop_flag = True
                break
                
    if outer_loop_flag == False:
        #print("This point is covered")
        return blurred_balls
    #else:
        #print("This point is not covered")
        
    K_union_A = list(K)
    K_union_A.extend(A)
    blurred_ball_new = approx_meb(K_union_A, epsilon / 3.0)
    
    discardables = []
    for blurred_ball in blurred_balls:
        lhs = blurred_ball.MEB.radius
        rhs = epsilon * blurred_ball_new.MEB.radius / 4.0
        #print("lhs <> rhs: " + repr(lhs) + " <> " + repr(rhs))
        if blurred_ball.MEB.radius <= (epsilon * blurred_ball_new.MEB.radius / 4.0):
            print "Found a discardable"
            discardables.append(blurred_ball)
    
    blurred_balls = [bb for bb in blurred_balls if bb not in discardables]
    
    blurred_balls.append(blurred_ball_new)
    
    return blurred_balls
        

In [81]:
def agarwal(points, num_of_points, num_of_dimensions):
    epsilon = 0.1
    batch_size = 100
    blurred_balls = []
    initial_points = points[0:num_of_dimensions, :]
    blurred_ball_init = approx_meb(initial_points, epsilon / 3.0)
    blurred_balls = [blurred_ball_init]

    for i in range(num_of_dimensions, num_of_points, batch_size):
        print "Iteration: " + repr(i)
        sid = i
        fid = min(i + batch_size, num_of_points)
        A = points[sid : fid, :]
        blurred_balls = update(blurred_balls, A, epsilon)
        
    return blurred_balls
  

In [82]:
def draw_blurred_balls(blurred_balls):
    xmin = min(points[:, 0])
    xmax = max(points[:, 0])
    ymin = min(points[:, 1])
    ymax = max(points[:, 1])

    print(xmin, xmax, ymin, ymax)
    # ball <- \phi
    ball = None
    count = 0
    processed_points = []
    fig, ax = plt.subplots(1, 1) 
    #ax.set_xlim(xmin - 10, xmax + 10)
    #ax.set_ylim(ymin - 10, ymax + 10)
    ax.set_aspect('equal', 'datalim')
    ax.set_xlim(xmin - 10, xmax + 10)
    ax.set_ylim(ymin - 10, ymax + 10)


    X = points[:, 0]
    Y = points[:, 1]
    scatters = plt.scatter(X, Y)

    for blurred_ball in blurred_balls:
        #print blurred_ball
        center, radius = blurred_ball.MEB
        circle = plt.Circle(center, radius, color='r', alpha = 0.1)
        ax.add_artist(circle)
        ax.add_artist(scatters)
    plt.show()

In [83]:
def draw_points_and_meb(points, meb):
    xmin = min(points[:, 0])
    xmax = max(points[:, 0])
    ymin = min(points[:, 1])
    ymax = max(points[:, 1])
    print(xmin, xmax, ymin, ymax)
    processed_points = []
    fig, ax = plt.subplots(1, 1) 
    #ax.set_xlim(xmin - 10, xmax + 10)
    #ax.set_ylim(ymin - 10, ymax + 10)
    ax.set_aspect('equal', 'datalim')
    ax.set_xlim(xmin - 50, xmax + 50)
    ax.set_ylim(ymin - 50, ymax + 50)
    X = points[:, 0]
    Y = points[:, 1]
    scatters = plt.scatter(X, Y)
    center = meb.center
    radius = meb.radius
    circle = plt.Circle(center, radius, color='r', alpha = 0.1)
    ax.add_artist(circle)

In [84]:
num_of_points = 10000
num_of_dimensions = 100
points = generate_normal_data(num_of_points, num_of_dimensions)
blurred_balls = agarwal(points, num_of_points, num_of_dimensions)
#draw_blurred_balls(blurred_balls)


Total Iterations: 60
Iteration: 100
update
Total Iterations: 60
Iteration: 200
update
Total Iterations: 60
Iteration: 300
update
Total Iterations: 60
Iteration: 400
update
Total Iterations: 60
Iteration: 500
update
Total Iterations: 60
Iteration: 600
update
Total Iterations: 60
Iteration: 700
update
Total Iterations: 60
Iteration: 800
update
Total Iterations: 60
Iteration: 900
update
Total Iterations: 60
Iteration: 1000
update
Total Iterations: 60
Iteration: 1100
update
Total Iterations: 60
Iteration: 1200
update
Total Iterations: 60
Iteration: 1300
update
Iteration: 1400
update
Total Iterations: 60
Iteration: 1500
update
Total Iterations: 60
Iteration: 1600
update
Total Iterations: 60
Iteration: 1700
update
Total Iterations: 60
Iteration: 1800
update
Total Iterations: 60
Iteration: 1900
update
Total Iterations: 60
Iteration: 2000
update
Total Iterations: 60
Iteration: 2100
update
Total Iterations: 60
Iteration: 2200
update
Total Iterations: 60
Iteration: 2300
update
Total Iterations: 

In [85]:
for i in range(len(blurred_balls)):
    print len(blurred_balls[i].k), blurred_balls[i].MEB.radius

16 34.2986610328
19 35.1320598719
18 35.8071147462
14 35.5344086102
16 35.8189515061
16 35.4864584757
22 36.2628636228
16 35.7273066493
20 36.5696397838
20 36.6385235271
21 36.6385235271
22 36.7517158811
22 36.7517158811
16 36.2598351032
18 36.8530432194
3 27.8178483815
3 27.8178483815
3 27.8178483815
3 27.8178483815
3 27.8178483815
3 27.8178483815
3 27.8178483815
3 27.8178483815
3 27.8178483815
3 27.8178483815
3 27.8178483815
3 27.8178483815
3 27.8178483815
3 27.8178483815
3 27.8178483815
3 27.8178483815
3 27.8178483815
3 27.8178483815
3 27.8178483815
3 27.8178483815
3 27.8178483815
3 27.8178483815
3 27.8178483815
3 27.8178483815
3 27.8178483815
3 27.8178483815
3 27.8178483815
3 27.8178483815
3 27.8178483815
3 27.8178483815
3 27.8178483815
3 27.8178483815
3 27.8178483815
3 27.8178483815
3 27.8178483815
3 27.8178483815
3 27.8178483815
3 27.8178483815
3 27.8178483815
3 27.8178483815
3 27.8178483815
3 27.8178483815
3 27.8178483815
3 27.8178483815
3 27.8178483815
3 27.8178483815
3 27.8178

In [86]:
import numpy as np
from sklearn.decomposition import PCA

def get_pca_transformer(points_hd):
    pca = PCA(n_components=2)
    pca.fit(points_hd)
    return pca

def get_blurred_ball_2d(blurred_ball, pca):
    points_hd = blurred_ball.k
    center_hd = blurred_ball.MEB.center
    points_hd.append(center_hd)
    
    points_2d = pca.fit_transform(points_hd)
    num_of_points = len(points_2d)
    center_2d = points_2d[num_of_points - 1, :]
    radius = blurred_ball.MEB.radius
    points_2d = points_2d[:num_of_points - 2, :]
    blurred_ball_2d = BlurredBall(points_2d, Ball(center_2d, radius))
    return blurred_ball_2d

In [87]:
'''
pca = get_pca_transformer(points)
blurred_balls_2d = [get_blurred_ball_2d(blurred_ball, pca) for blurred_ball in blurred_balls]
print(blurred_balls_2d[0])
draw_blurred_balls(blurred_balls_2d)
'''

'\npca = get_pca_transformer(points)\nblurred_balls_2d = [get_blurred_ball_2d(blurred_ball, pca) for blurred_ball in blurred_balls]\nprint(blurred_balls_2d[0])\ndraw_blurred_balls(blurred_balls_2d)\n'

# Timothy Chan's Algorithm

In [88]:
Ball = namedtuple("Ball", ["center", "radius"])

def inside_ball(ball, point):
    center, radius = ball
    distance = npl.norm(point - center)
    if distance < radius:
        return True
    else:
        return False

def meb_ball_and_point(ball, p):
    c, r = ball
    pc_scalar = npl.norm(c - p)
    pc_vector = c - p
    radius_unit = pc_vector / pc_scalar
    p_mirror = radius_unit * r + c
    c_prime = (p + p_mirror) / 2.0
    r_prime = npl.norm(p_mirror - p) / 2.0
    meb = Ball(c_prime, r_prime)
    return meb

def create_initial_ball(point):
    return Ball(point, 0.0)

def chan(points):
    count = 0
    center = None
    radius = None
    ball = None
    for point in points:
        if ball == None:
            ball = create_initial_ball(point)
            continue
        if inside_ball(ball, point):
            continue
        else:
            count += 1
            ball = meb_ball_and_point(ball, point)
            center, radius = ball
    return Ball(center, radius)

In [89]:
meb = chan(points)
print(meb)
    

Ball(center=array([ 93.25112249,  68.72477846,  28.35796716,  72.18554431,
        64.29257589,  49.71006458,  50.96422265,  44.04482468,
        91.21222664,  23.40102818,  71.14704989,  44.96417518,
        41.40570617,  72.69668638,  32.01735859,   5.88414597,
        69.53903002,  -0.37792451,  78.06422461,  31.60418348,
        58.97742347,  60.38535503,   9.65871452,  51.64135807,
         9.48328565,  35.10441453,  25.83608534,  98.6366631 ,
         4.50705628,  17.50536644,  12.42156074,  83.17156213,
         6.48630908,  11.6394207 ,  66.84924632,  83.95017241,
        97.16381511,  73.59301791,  62.1522737 ,  47.6508593 ,
        87.65115847,  96.98787025,  37.25992057,  34.74636041,
        64.86903178,  76.77719871,  81.39296406,  23.30220288,
        24.26040391,   3.50451708,  34.63007344,  78.86407583,
        29.58557539,  18.97479922,  43.33361164,  58.07433625,
        39.90832393,  85.00878129,  61.58898805,   6.73642543,
        99.61900372,  56.38101061,  34.1592